In [32]:
import sys
import numpy as np
import pandas as pd

import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline


import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model


from keras.layers.pooling import MaxPooling2D

from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform


from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [33]:
import os
from os import listdir

In [34]:
train = "/workspace/Shantan_Data/Bayesian2v/Steel_Surface/data1/train6/"
val = "/workspace/Shantan_Data/Bayesian2v/Steel_Surface/data1/val4/"
save = "/workspace/Shantan_Data/Bayesian2v/Steel_Surface/data1/save"

In [35]:

from PIL import Image

In [36]:
def loadimgs(path,n = 0):
    '''
    path = "/workspace/Shantan_Data/Bayesian2v/Steel_Surface/data1/train6/"
    
    '''
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    # we load every alphabet seperately so we can isolate them later
    for fname in os.listdir(path):
        print("loading fname: " + fname)
        lang_dict[fname] = [curr_y,None]
        fname_path = os.path.join(path,fname)
        # every letter/category has it's own column in the array, so  load seperately
        for subf in os.listdir(fname_path):
            cat_dict[curr_y] = (fname, subf)
            category_images=[]
            subf_path = os.path.join(fname_path, subf)
            # read all the images in the current category
            for i in os.listdir(subf_path):
                image_path = os.path.join(subf_path, i)
                image = Image.open(image_path)
                image = image.resize((105, 105))
                category_images.append(image)
                y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            lang_dict[fname][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict

In [37]:
X,y,c=loadimgs(train)

loading fname: defect2
need at least one array to stack
error - category_images: []
loading fname: defect4
need at least one array to stack
error - category_images: []
loading fname: defect6
need at least one array to stack
error - category_images: []
loading fname: defect5
need at least one array to stack
error - category_images: []
loading fname: defect3
need at least one array to stack
error - category_images: []
loading fname: defect1
need at least one array to stack
error - category_images: []


In [38]:
X.shape

(6, 50, 105, 105)

In [39]:
c

{'defect2': [0, 1],
 'defect4': [2, 3],
 'defect6': [4, 5],
 'defect5': [6, 7],
 'defect3': [8, 9],
 'defect1': [10, 11]}

In [40]:
with open(os.path.join(save,"train.pickle"), "wb") as f:
    pickle.dump((X,c),f)

In [41]:
Xval,yval,cval=loadimgs(val)

loading fname: defect8
need at least one array to stack
error - category_images: []
loading fname: defect9
need at least one array to stack
error - category_images: []
loading fname: defect10
need at least one array to stack
error - category_images: []
loading fname: defect7
need at least one array to stack
error - category_images: []


In [42]:
with open(os.path.join(save,"val.pickle"), "wb") as f:
    pickle.dump((Xval,cval),f)

In [43]:
def initialize_weights(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [44]:
def initialize_bias(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [45]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [46]:
model = get_siamese_model((105, 105,1))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 105, 105, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 105, 105, 1  0           []                               
                                )]                                                                
                                                                                                  
 sequential_1 (Sequential)      (None, 4096)         38947648    ['input_3[0][0]',                
                                                                  'input_4[0][0]']          

In [47]:
optimizer = Adam(lr =1e-5)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [48]:
X.shape

(6, 50, 105, 105)

In [49]:
with open(os.path.join(save, "train.pickle"), "rb") as f:
    (Xtrain, train_classes) = pickle.load(f)
    
print("Training Categories: \n")
print(list(train_classes.keys()))

Training Categories: 

['defect2', 'defect4', 'defect6', 'defect5', 'defect3', 'defect1']


In [50]:
with open(os.path.join(save, "val.pickle"), "rb") as f:
    (Xval, val_classes) = pickle.load(f)

print("Validation Categories:", end="\n\n")
print(list(val_classes.keys()))

Validation Categories:

['defect8', 'defect9', 'defect10', 'defect7']


In [51]:
Xval.shape

(4, 30, 105, 105)

In [52]:
train_classes

{'defect2': [0, 1],
 'defect4': [2, 3],
 'defect6': [4, 5],
 'defect5': [6, 7],
 'defect3': [8, 9],
 'defect1': [10, 11]}

In [53]:
val_classes

{'defect8': [0, 1], 'defect9': [2, 3], 'defect10': [4, 5], 'defect7': [6, 7]}

In [54]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if s == 'train':
        X = Xtrain
        categories = train_classes
    
    else:
        X = Xval
        categories = val_classes
    
    
    n_classes, n_examples, w, h = X.shape
    categories = rng.choice(n_classes,size=(batch_size,), replace=True)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(h, w, 1)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(h, w,1)
    
    return pairs, targets

In [55]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [56]:
def make_oneshot_task(N, s="val", fname=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
        
    n_classes, n_examples, w, h = X.shape
    print(n_classes)
    indices = rng.randint(0, n_examples,size=(N,))
    if fname is not None: # if language is specified, select characters for that language
        low, high = categories[fname]
        if N > high - low:
            raise ValueError("This fname ({}) has less than {} class".format(fname, N))
        categories = rng.choice(range(low,high),size=(N,), replace=True)

    else: # if no fname specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,), replace=True)            
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples, replace=True, size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

In [57]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [58]:
#hyperparameters
evaluate_every = 500 # interval for evaluating on one-shot tasks
batch_size = 28
n_iter = 25000 # No. of training iterations
N_way = 4 # how many classes for testing one-shot tasks
n_val = 120 # how many one-shot tasks to validate on
best = -1

In [59]:
model_path = "/workspace/Shantan_Data/Bayesian2v/Steel_Surface/data1/model/"

In [61]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------

 ------------- 

Time for 500 iterations: 0.29007641871770223 mins
Train Loss: 0.07586399465799332
Evaluating model on 120 random 4 way one-shot learning tasks ... 

4
1/1 [==============================] - 0s 30ms/step
4
1/1 [==============================] - 0s 28ms/step
4
1/1 [==============================] - 0s 28ms/step
4
1/1 [==============================] - 0s 28ms/step
4
1/1 [==============================] - 0s 28ms/step
4
1/1 [==============================] - 0s 29ms/step
4
1/1 [==============================] - 0s 29ms/step
4
1/1 [==============================] - 0s 27ms/step
4
1/1 [==============================] - 0s 31ms/step
4
1/1 [==============================] - 0s 28ms/step
4
1/1 [==============================] - 0s 29ms/step
4
1/1 [==============================] - 0s 30ms/step
4
1/1 [==============================] - 0s 29ms/step
4
1/1 [==============================] - 0s 28ms/step
4
1/1 [=====

In [62]:
ways = np.arange(1,5)
resume =  False
trials = 100

In [63]:
val_accs, train_accs,nn_accs = [], [], []
for N in ways:    
    val_accs.append(test_oneshot(model, N, trials, "val", verbose=True))
    train_accs.append(test_oneshot(model, N, trials, "train", verbose=True))
#     nn_acc = test_nn_accuracy(N, trials)
#     nn_accs.append(nn_acc)
#     print ("NN Accuracy = ", nn_acc)
    print("------------------------------------------------------------------------------------------------")

Evaluating model on 100 random 1 way one-shot learning tasks ... 

4
1/1 [==============================] - 0s 43ms/step
4
1/1 [==============================] - 0s 40ms/step
4
1/1 [==============================] - 0s 40ms/step
4
1/1 [==============================] - 0s 40ms/step
4
1/1 [==============================] - 0s 39ms/step
4
1/1 [==============================] - 0s 39ms/step
4
1/1 [==============================] - 0s 40ms/step
4
1/1 [==============================] - 0s 40ms/step
4
1/1 [==============================] - 0s 40ms/step
4
1/1 [==============================] - 0s 39ms/step
4
1/1 [==============================] - 0s 39ms/step
4
1/1 [==============================] - 0s 44ms/step
4
1/1 [==============================] - 0s 41ms/step
4
1/1 [==============================] - 0s 44ms/step
4
1/1 [==============================] - 0s 60ms/step
4
1/1 [==============================] - 0s 47ms/step
4
1/1 [==============================] - 0s 119ms/step
4
1/1 [=======

In [ ]:
# from keras.layers import Concatenate

In [ ]:
# def concat_images(X):
#     """Concatenates a bunch of images into a big matrix for plotting purposes."""
#     nc, h , w, _ = X.shape
#     X = X.reshape(nc, h, w)
#     n = np.ceil(np.sqrt(nc)).astype("int8")
#     img = np.zeros((n*w,n*h))
#     x = 0
#     y = 0
#     for example in range(nc):
#         img[x*w:(x+1)*w,y*h:(y+1)*h] = X[example]
#         y += 1
#         if y >= n:
#             y = 0
# #             x += 1
#     return img

In [ ]:
# def plot_oneshot_task(pairs):
#     fig,(ax1,ax2) = plt.subplots(nrows=1, ncols=2)
#     ax1.matshow(pairs[0][0].reshape(105,105), cmap='gray')
#     img = concat_images(pairs[1])
#     ax1.get_yaxis().set_visible(False)
#     ax1.get_xaxis().set_visible(False)
#     ax2.matshow(img,cmap='gray')
#     plt.xticks([])
#     plt.yticks([])
#     plt.show()

In [ ]:
# pairs, targets = make_oneshot_task(1,"train","defect1")
# plot_oneshot_task(pairs)